In [2]:
from sklearn.datasets import fetch_olivetti_faces
import tensorflow as tf
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_olivetti_faces
import os
import numpy as np
from sklearn import preprocessing


C:\Users\SONY\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
faces=fetch_olivetti_faces()

In [4]:
import numpy as np 
import matplotlib.pyplot as plt

In [5]:
data=faces
targets=faces.target
data = data.images.reshape((len(data.images), -1))

In [6]:
lab=preprocessing.LabelBinarizer()
targets=lab.fit_transform(targets)
print(targets.shape)
print(targets[0])

(400, 40)
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]


In [7]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(data, targets, test_size=0.1, random_state=0)

In [8]:
x = tf.placeholder(tf.float32, [None, 4096])
y=tf.placeholder(tf.float32, [None,40])
W = tf.Variable(tf.zeros([4096,40]))
b = tf.Variable(tf.zeros([40]))


In [9]:
learning_rate=0.001
training_epochs=350
batch_size=100


In [10]:
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
logs_path = 'H:\\tmp\\tensorflow_logs\\xample1.7'

In [11]:
with tf.name_scope("Model"):
    pred=tf.nn.softmax(tf.matmul(x,W)+b)
with tf.name_scope("Loss"):
    cost=tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred),reduction_indices=1))
with tf.name_scope("SGD"):
    optimizer=tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
with tf.name_scope("Accuracy"):
    correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [12]:

init=tf.global_variables_initializer()


In [13]:
tf.summary.histogram("loss", cost)
tf.summary.scalar("loss", cost)
tf.summary.scalar("accuracy", accuracy)
tf.summary.histogram("accuracy", accuracy)
merged_summary_op = tf.summary.merge_all()

In [14]:
with tf.Session() as sess:
    sess.run(init)
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    for epoch in range(training_epochs):
        avg_cost=0
        total_batch=int(len(xtrain)/batch_size)
        for i in range(total_batch):
            _,c,summary=sess.run([optimizer,cost,merged_summary_op], feed_dict={x:xtrain,y:ytrain})
            summary_writer.add_summary(summary,total_batch*epoch+i)
            avg_cost+=c/total_batch

        #print("Epoch:",epoch,"cost =",avg_cost)
    print("Training accuracy",accuracy.eval({x:xtrain,y:ytrain})*100)
    print("Test accuracy",accuracy.eval({x:xtest,y:ytest})*100)

Training accuracy 78.61111164093018
Test accuracy 55.000001192092896
